# TF-IDF From Scratch

Library Import

In [72]:
import pandas as pd
import numpy as np
import nltk
import string

from nltk import word_tokenize
from scipy.sparse import csr_matrix

In [2]:
nltk.download('punkt')

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\user\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

## Data Loading

Note that dataset can be found here: https://www.kaggle.com/tmdb/tmdb-movie-metadata

Read a csv file.

In [3]:
path = "tmdb_5000_movies.csv"
df = pd.read_csv(path)
df

,budget,genres,homepage,id,keywords,original_language,original_title,overview,popularity,production_companies,production_countries,release_date,revenue,runtime,spoken_languages,status,tagline,title,vote_average,vote_count
0,237000000,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...",http://www.avatarmovie.com/,19995,"[{""id"": 1463, ""name"": ""culture clash""}, {""id"":...",en,Avatar,"In the 22nd century, a paraplegic Marine is di...",150.437577,"[{""name"": ""Ingenious Film Partners"", ""id"": 289...","[{""iso_3166_1"": ""US"", ""name"": ""United States o...",2009-12-10,2787965087,162.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}, {""iso...",Released,Enter the World of Pandora.,Avatar,7.2,11800
1,300000000,"[{""id"": 12, ""name"": ""Adventure""}, {""id"": 14, ""...",http://disney.go.com/disneypictures/pirates/,285,"[{""id"": 270, ""name"": ""ocean""}, {""id"": 726, ""na...",en,Pirates of the Caribbean: At World's End,"Captain Barbossa, long believed to be dead, ha...",139.082615,"[{""name"": ""Walt Disney Pictures"", ""id"": 2}, {""...","[{""iso_3166_1"": ""US"", ""name"": ""United States o...",2007-05-19,961000000,169.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,"At the end of the world, the adventure begins.",Pirates of the Caribbean: At World's End,6.9,4500
2,245000000,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...",http://www.sonypictures.com/movies/spectre/,206647,"[{""id"": 470, ""name"": ""spy""}, {""id"": 818, ""name...",en,Spectre,A cryptic message from Bond’s past sends him o...,107.376788,"[{""name"": ""Columbia Pictures"", ""id"": 5}, {""nam...","[{""iso_3166_1"": ""GB"", ""name"": ""United Kingdom""...",2015-10-26,880674609,148.0,"[{""iso_639_1"": ""fr"", ""name"": ""Fran\u00e7ais""},...",Released,A Plan No One Escapes,Spectre,6.3,4466
3,250000000,"[{""id"": 28, ""name"": ""Action""}, {""id"": 80, ""nam...",http://www.thedarkknightrises.com/,49026,"[{""id"": 849, ""name"": ""dc comics""}, {""id"": 853,...",en,The Dark Knight Rises,Following the death of District Attorney Harve...,112.312950,"[{""name"": ""Legendary Pictures"", ""id"": 923}, {""...","[{""iso_3166_1"": ""US"", ""name"": ""United States o...",2012-07-16,1084939099,165.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,The Legend Ends,The Dark Knight Rises,7.6,9106
4,260000000,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...",http://movies.disney.com/john-carter,49529,"[{""id"": 818, ""name"": ""based on novel""}, {""id"":...",en,John Carter,"John Carter is a war-weary, former military ca...",43.926995,"[{""name"": ""Walt Disney Pictures"", ""id"": 2}]","[{""iso_3166_1"": ""US"", ""name"": ""United States o...",2012-03-07,284139100,132.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,"Lost in our world, found in another.",John Carter,6.1,2124
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4798,220000,"[{""id"": 28, ""name"": ""Action""}, {""id"": 80, ""nam...",NaN,9367,"[{""id"": 5616, ""name"": ""united states\u2013mexi...",es,El Mariachi,El Mariachi just wants to play his guitar and ...,14.269792,"[{""name"": ""Columbia Pictures"", ""id"": 5}]","[{""iso_3166_1"": ""MX"", ""name"": ""Mexico""}, {""iso...",1992-09-04,2040920,81.0,"[{""iso_639_1"": ""es"", ""name"": ""Espa\u00f1ol""}]",Released,"He didn't come looking for trouble, but troubl...",El Mariachi,6.6,238
4799,9000,"[{""id"": 35, ""name"": ""Comedy""}, {""id"": 10749, ""...",NaN,72766,[],en,Newlyweds,A newlywed couple's honeymoon is upended by th...,0.642552,[],[],2011-12-26,0,85.0,[],Released,A newlywed couple's honeymoon is upended by th...,Newlyweds,5.9,5
4800,0,"[{""id"": 35, ""name"": ""Comedy""}, {""id"": 18, ""nam...",http://www.hallmarkchannel.com/signedsealeddel...,231617,"[{""id"": 248, ""name"": ""date""}, {""id"": 699, ""nam...",en,"Signed, Sealed, Delivered","""Signed, Sealed, Delivered"" introduces a dedic...",1.444476,"[{""name"": ""Front Street Pictures"", ""

List all fields

In [4]:
id_field = "id"
budget_field = "budget"
genres_field = "genres"
homepage_field = "homepage"
keyword_field = "keywords"
original_language_field = "original_language"
original_title_field = "original_title"
overview_field = "overview"
popularity_field = "popularity"
production_companies_field = "production_companies"
production_countries_field = "production_countries"
release_date_field = "release_date"
revenue_field = "revenue"
runtime_field = "runtime"
spoken_languages_field = "spoken_languages"
status_field = "status"
tagline_field = "tagline"
title_field = "title"
vote_average_field = "vote_average"
vote_count_field = "vote_count"

Define Fields

Note that we also remove unecessary fields

In [5]:
fields = df.columns.to_list()
target = original_title_field

fields.remove(id_field)
fields.remove(homepage_field)
fields.remove(title_field)
fields.remove(original_title_field)
fields.remove(revenue_field)
fields.remove(vote_count_field)

print(fields)

['budget', 'genres', 'keywords', 'original_language', 'overview', 'popularity', 'production_companies', 'production_countries', 'release_date', 'runtime', 'spoken_languages', 'status', 'tagline', 'vote_average']


In [6]:
df[fields].info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4803 entries, 0 to 4802
Data columns (total 14 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   budget                4803 non-null   int64  
 1   genres                4803 non-null   object 
 2   keywords              4803 non-null   object 
 3   original_language     4803 non-null   object 
 4   overview              4800 non-null   object 
 5   popularity            4803 non-null   float64
 6   production_companies  4803 non-null   object 
 7   production_countries  4803 non-null   object 
 8   release_date          4802 non-null   object 
 9   runtime               4801 non-null   float64
 10  spoken_languages      4803 non-null   object 
 11  status                4803 non-null   object 
 12  tagline               3959 non-null   object 
 13  vote_average          4803 non-null   float64
dtypes: float64(3), int64(1), object(10)
memory usage: 525.5+ KB


In [7]:
df[fields].describe()

,budget,popularity,runtime,vote_average
count,4.803000e+03,4803.000000,4801.000000,4803.000000
mean,2.904504e+07,21.492301,106.875859,6.092172
std,4.072239e+07,31.816650,22.611935,1.194612
min,0.000000e+00,0.000000,0.000000,0.000000
25%,7.900000e+05,4.668070,94.000000,5.600000
50%,1.500000e+07,12.921594,103.000000,6.200000
75%,4.000000e+07,28.313505,118.000000,6.800000
max,3.800000e+08,875.581305,338.000000,10.000000


## Data Preprocessing

### Convert Number Fields to Word

In [15]:
def budget_to_string(row):
    if row >= 5000000:
        return "expensive"
    else:
        return "cheap"

def popularity_to_string(row):
    if row >= 12:
        return "Popular"
    else:
        return "Rare"

def runtime_to_string(row):
    if row >= 94:
        return "Long"
    else:
        return "Short"

def vote_average_to_string(row):
    if row >= 7.5:
        return "Good"
    elif row > 5.6 and row < 7.5:
        return "Average"
    else:
        return "Poor"

number_to_word_converted_df = df[fields].copy()

number_to_word_converted_df[budget_field] = number_to_word_converted_df[budget_field].apply(budget_to_string)
number_to_word_converted_df[popularity_field] = number_to_word_converted_df[popularity_field].apply(popularity_to_string)
number_to_word_converted_df[runtime_field] = number_to_word_converted_df[runtime_field].apply(runtime_to_string)
number_to_word_converted_df[vote_average_field] = number_to_word_converted_df[vote_average_field].apply(vote_average_to_string)

number_to_word_converted_df.head(3)

,budget,genres,keywords,original_language,overview,popularity,production_companies,production_countries,release_date,runtime,spoken_languages,status,tagline,vote_average
0,expensive,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...","[{""id"": 1463, ""name"": ""culture clash""}, {""id"":...",en,"In the 22nd century, a paraplegic Marine is di...",Popular,"[{""name"": ""Ingenious Film Partners"", ""id"": 289...","[{""iso_3166_1"": ""US"", ""name"": ""United States o...",2009-12-10,Long,"[{""iso_639_1"": ""en"", ""name"": ""English""}, {""iso...",Released,Enter the World of Pandora.,Average
1,expensive,"[{""id"": 12, ""name"": ""Adventure""}, {""id"": 14, ""...","[{""id"": 270, ""name"": ""ocean""}, {""id"": 726, ""na...",en,"Captain Barbossa, long believed to be dead, ha...",Popular,"[{""name"": ""Walt Disney Pictures"", ""id"": 2}, {""...","[{""iso_3166_1"": ""US"", ""name"": ""United States o...",2007-05-19,Long,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,"At the end of the world, the adventure begins.",Average
2,expensive,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...","[{""id"": 470, ""name"": ""spy""}, {""id"": 818, ""name...",en,A cryptic message from Bond’s past sends him o...,Popular,"[{""name"": ""Columbia Pictures"", ""id"": 5}, {""nam...","[{""iso_3166_1"": ""GB"", ""name"": ""United Kingdom""...",2015-10-26,Long,"[{""iso_639_1"": ""fr"", ""name"": ""Fran\u00e7ais""},...",Released,A Plan No One Escapes,Average


### Convert JSON in fields of Movie Dataset into string

In [9]:
def convert_json_field_in_df_into_strings(df, df_field, json_field):
    import json
    
    """
    Convert JSON fields to string, by taking the dataframe and a field, a json field, which returns a list of string

    from a specific field in a dataframe

    Below are the explanation of parameters used.
    * df: dataframe to be used
    * df_field: field in dataframe, to convert json into strings
    * json_field: one field to be kept from json inside the data field in dataframe.

    Consider the example:

    | id | movie_name | movie_genres |

    | 1 | a           | {"id": 1, "name": "action"}, {"id": 2, "name": "romance"} | 

    | 2 | b           | {"id": 1, "name": "slice of life"}, {"id": 2, "name": "food"} |

    If we use the function: 
    
    convert_json_field_in_df_into_strings(df, "movie_genres", "name")

    This gives us the list of string from all rows in movie_genres field.

    """

    json_data = json.loads(df[df_field].to_json())
    json_values = list(json_data.values())

    list_of_strings = []

    for value in json_values:

        my_string = ""

        dicts = list(json.loads(value))[:]

        for dict_value in dicts:
            my_string += f"{dict_value[json_field]} "
        list_of_strings.append(my_string[:-1])

    return list_of_strings

In [16]:
json_converted_df = number_to_word_converted_df.copy()

json_field = "name"

json_converted_df[genres_field] = convert_json_field_in_df_into_strings(json_converted_df, genres_field, json_field)
json_converted_df[keyword_field] = convert_json_field_in_df_into_strings(json_converted_df, keyword_field, json_field)
json_converted_df[spoken_languages_field] = convert_json_field_in_df_into_strings(json_converted_df, spoken_languages_field, json_field)
json_converted_df[production_companies_field] = convert_json_field_in_df_into_strings(json_converted_df, production_companies_field, json_field)
json_converted_df[production_countries_field] = convert_json_field_in_df_into_strings(json_converted_df, production_countries_field, json_field)

json_converted_df.head(3)

,budget,genres,keywords,original_language,overview,popularity,production_companies,production_countries,release_date,runtime,spoken_languages,status,tagline,vote_average
0,expensive,Action Adventure Fantasy Science Fiction,culture clash future space war space colony so...,en,"In the 22nd century, a paraplegic Marine is di...",Popular,Ingenious Film Partners Twentieth Century Fox ...,United States of America United Kingdom,2009-12-10,Long,English Español,Released,Enter the World of Pandora.,Average
1,expensive,Adventure Fantasy Action,ocean drug abuse exotic island east india trad...,en,"Captain Barbossa, long believed to be dead, ha...",Popular,Walt Disney Pictures Jerry Bruckheimer Films S...,United States of America,2007-05-19,Long,English,Released,"At the end of the world, the adventure begins.",Average
2,expensive,Action Adventure Crime,spy based on novel secret agent sequel mi6 bri...,en,A cryptic message from Bond’s past sends him o...,Popular,Columbia Pictures Danjaq B24,United Kingdom United States of America,2015-10-26,Long,Français English Español Italiano Deutsch,Released,A Plan No One Escapes,Average


### Combine all df rows and columns into array of strings.

In [12]:
def combine_all_strings_from_df(df, fields, whitespace_split = ','):
    """
    Combine all strings from one df and all fields. 
    
    If you'd like to put something between words, provide whitespace_split parameter.
    """
    df_strings = df.to_string(header = False,
                       index = False,
                       columns = fields,
                       index_names = False).split('\n')

    all_strings = [" ".join(s.split()) for s in df_strings]

    return all_strings

df_strings = combine_all_strings_from_df(json_converted_df, fields)

print(df_strings[0])

expensive Action Adventure Fantasy Science Fiction culture clash future space war space colony society space travel futuristic romance space alien tribe alien planet cgi marine soldier battle love affair anti war power relations mind and soul 3d en In the 22nd century, a paraplegic Marine is dispatched to the moon Pandora on a unique mission, but becomes torn between following orders and protecting an alien civilization. Popular Ingenious Film Partners Twentieth Century Fox Film Corporation Dune Entertainment Lightstorm Entertainment United States of America United Kingdom 2009-12-10 Long English Español Released Enter the World of Pandora. Average


In [30]:
def get_erased_punctuations(text):
    list_of_punctuations = string.punctuation

    final_text = text
    for punc in list_of_punctuations:
        final_text = final_text.replace(punc, "")

    return final_text

erased_punc_df_strings = [get_erased_punctuations(my_string) for my_string in df_strings]
erased_punc_df_strings[0]

'expensive Action Adventure Fantasy Science Fiction culture clash future space war space colony society space travel futuristic romance space alien tribe alien planet cgi marine soldier battle love affair anti war power relations mind and soul 3d en In the 22nd century a paraplegic Marine is dispatched to the moon Pandora on a unique mission but becomes torn between following orders and protecting an alien civilization Popular Ingenious Film Partners Twentieth Century Fox Film Corporation Dune Entertainment Lightstorm Entertainment United States of America United Kingdom 20091210 Long English Español Released Enter the World of Pandora Average'

## TF-IDF

### Word to Index Mapping

Populate into text mapping, so we know which word maps to which columns in our TF-IDF matrix.

In [59]:
def convert_set_to_indexed_dict(current_set):
    indexed_dict = {}

    for index, value in enumerate(current_set):
        indexed_dict[index] = value
    
    return indexed_dict

def word_to_index_mapping(list_of_documents):
    all_unique_tokenized_words = set()

    for d in list_of_documents:
        tokenized_sentences = word_tokenize(d)
        all_unique_tokenized_words.update(tokenized_sentences)

    word_to_index_mapping = convert_set_to_indexed_dict(all_unique_tokenized_words)
    return word_to_index_mapping

def dict_head(dictionary, max_index = 5):
    print(list(dictionary.items())[:max_index])

df_word_to_index_dict = word_to_index_mapping(erased_punc_df_strings)

dict_head(df_word_to_index_dict, 3)

[(0, 'Think'), (1, 'swordfishing'), (2, 'recounts')]


### Count Vectorizer

In [87]:
def custom_count_vectorizer(unique_words, documents):
    unique_values_to_count = list(unique_words.values())
    
    total_words_in_documents = np.array([], dtype = np.int0)

    for d in documents:
        words = word_tokenize(d)
        total_words_in_documents = np.append(total_words_in_documents, [words.count(u) for u in unique_values_to_count])
        break
    
    print(print(total_words_in_documents))

custom_count_vectorizer(df_word_to_index_dict, erased_punc_df_strings)

[0 0 0 ... 0 0 0]
None
